# Reto 03-B-Chunking (Segmentación)

## 1. Descripción General 

En este notebook, repasarás los conceptos de tokens y chunking. En el notebook anterior (`CH-03-A-Grounding`), pudimos proporcionar algo de contexto adicional para fundamentar el modelo. ¿Existe un límite para la cantidad de contexto adicional que podemos proporcionar al modelo? Desafortunadamente, la respuesta es sí. Existe un límite para la cantidad de tokens permitidos en la entrada y la salida combinadas según el modelo que se esté utilizando.

Entonces, ¿qué son los tokens? Los tokens son una representación de cómo los modelos de Azure OpenAI procesan el texto. Son palabras o simplemente fragmentos de caracteres. Veamos el número total de tokens en la respuesta que obtuvimos del primer notebook del Desafío 3. Hay muchas formas de calcular tokens. En este notebook, echaremos un vistazo a la biblioteca `tiktoken` para contar los tokens.

## 2. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos de configuración de claves que ya completaste en los desafíos anteriores.

In [1]:
! pip install --upgrade click
! python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import openai
import PyPDF3
import os
import json
import tiktoken
import spacy
from openai.error import InvalidRequestError

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from spacy.lang.en import English 
nlp = spacy.load("en_core_web_sm")

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [3]:
# Load your OpenAI credentials
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
model=os.getenv("CHAT_MODEL_NAME")


## 3. Conteo de Tokens

`tiktoken` utiliza una técnica llamada codificación de pares de bytes (BPE) para convertir el texto dado en tokens. Hay diferentes codificaciones disponibles para ayudar a procesar las palabras. En este norebook, utilizaremos `cl100k_base`.

#### Tarea #1 del Estudiante: 

Cuenta el número de tokens en la respuesta final que recibimos en `CH-03-A-Grounding` completando la función `count_tokens` a continuación.

In [4]:

def count_tokens(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
count_tokens("tiktoken is great!", "cl100k_base")

6

#### Tarea #2 del Estudiante:

Ingresa el texto de la respuesta que recibiste en `CH-03-A-Grounding`. Ejecuta la celda a continuación para recuperar el número de tokens utilizando la función `count_tokens`.

In [6]:
text = """El ganador del torneo de individuales masculino de Wimbledon 2023 fue Carlos Alcaraz, mientras que la ganadora del torneo de individuales femenino fue Markéta Vondroušová.
"""

count_tokens(text, "cl100k_base")

print("There are " + str(count_tokens(text, "cl100k_base")) + " tokens: " + text)

There are 47 tokens: El ganador del torneo de individuales masculino de Wimbledon 2023 fue Carlos Alcaraz, mientras que la ganadora del torneo de individuales femenino fue Markéta Vondroušová.



Muy bien, ahora sabemos con cuántos tokens estamos trabajando. ¿Qué sucede si queremos agregar más contexto que el que ya pusimos en la variable de texto anterior? Si pensamos en nuestro escenario de Wimbledon, necesitaremos darle al modelo más contexto para ayudarlo a entender todo lo que necesita saber sobre el torneo. Más importante aún, todo lo que necesita saber para ayudar a responder tus preguntas al escribir el informe. Digamos que queremos proporcionar más contexto al modelo con un documento PDF. ¿Podemos intentar obtener un resumen del documento PDF para ayudarnos con nuestro trabajo?

#### Tarea #3 del Estudiante:

En la celda siguiente, inserta la ruta del documento PDF, `CH3-data.pdf`, que se encuentra en la carpeta `/data` proporcionada. Ejecuta las tres celdas para ver la salida.

In [7]:
document = open(r'../data/CH3-data.pdf', 'rb') 
doc_helper = PyPDF3.PdfFileReader(document)

In [8]:
finaltext = ''
totalpages = doc_helper.getNumPages()
for eachpage in range(totalpages):
   p = doc_helper.getPage(eachpage)
   indpagetext = p.extractText()
   finaltext += indpagetext

clean_text = finaltext.replace("  ", " ").replace("\n", "; ").replace(';',' ')

In [9]:
prompt = f"What is the answer to the following question regarding the PDF document?\n\n{finaltext}\n\n" 
q = "Can you give me a summary of the document?"

try:
    final_prompt = prompt + q
    response = openai.ChatCompletion.create(engine=model, messages=final_prompt, max_tokens=50)
    answer = response.choices[0].text.strip()
    print(f"{q}\n{answer}\n")

except InvalidRequestError as e:
    print(e.error)



{
  "code": null,
  "message": "'What is the answer to the following question regarding the PDF document?\\n\\n \\nFormula 1 Power Unit Financial Regulations\\n \\n1\\n \\n16 August\\n \\n2022\\n \\n\u00a9 202\\n2\\n \\n\\n \\n \\n \\nIssue \\n1\\n \\n \\n \\n \\nFORMULA 1 \\nPOWER UNIT \\nFINANCIAL REGULATIONS\\n \\nPUBLISHED ON \\n16 August\\n \\n2022\\n \\nIssue \\n1\\n \\n \\nCONTENTS\\n \\n \\nArt\\n \\nCONTENTS\\n \\nPage(s)\\n \\n1.\\n \\nGENERAL PRINCIPLES\\n \\n................................\\n................................\\n................................\\n........\\n \\n2\\n \\n2.\\n \\nPOWER UNIT MANUFACTURER OBLIGATIONS\\n \\n................................\\n................................\\n....\\n \\n3\\n \\n3.\\n \\nEXCLUSIONS\\n \\n................................\\n................................\\n................................\\n.....................\\n \\n5\\n \\n4.\\n \\nADJUSTMENTS\\n \\n................................\\n............................

Como has podido observar, recibiste un mensaje de error después de ejecutar el fragmento de código anterior. El modelo alcanza su longitud de contexto máxima. Para los modelos GPT-3, el límite de tokens es de 4097 tokens. ¿Cómo solucionamos este problema al proporcionarle todo el contexto necesario, pero sin encontrarnos con el problema del límite de tokens?

Para resolver este problema, podemos echar un vistazo a un concepto llamado Chunking (Segmentación).

## 4. Chunking (Segmentación)

La segmentación ayuda a limitar la cantidad de información que pasamos al modelo. La información que pasaremos son los trozos más relevantes de los datos en general. Hay muchas consideraciones que entran en juego al hacer segmentación. Por ejemplo, necesitas averiguar el mejor tamaño de chunk (trozo). Si los trozos son demasiado pequeños, puedes perder contexto importante. Si los trozos son demasiado grandes, pueden contener información innecesaria.

A continuación se presentan algunas técnicas comunes de chunking.

1. Chunking con trozos más pequeños
2. Chunking dividiendo las oraciones
3. Chunking con superposición de oraciones
4. Chunking de forma recursiva

Veamos estas técnicas en acción.

### 4.1 Chunking con trozos más pequeños

#### Tarea #4 del Estudiante: 

Agrega código en la celda siguiente. Utiliza la función `split()` para dividir el texto en trozos.

In [10]:

text = "The sun was setting over the horizon, casting a warm glow over the landscape. Birds chirped in the trees, and a gentle breeze rustled the leaves. In the distance, a herd of deer grazed in a meadow. The air was filled with the sweet scent of blooming flowers. It was a peaceful and serene scene, perfect for a quiet evening stroll."

# Agrega tu código aquí # buscar sobre la funcion split() en google o chatgpt o w3school...

x = text.split()

for ch in x:
    print(ch)


The
sun
was
setting
over
the
horizon,
casting
a
warm
glow
over
the
landscape.
Birds
chirped
in
the
trees,
and
a
gentle
breeze
rustled
the
leaves.
In
the
distance,
a
herd
of
deer
grazed
in
a
meadow.
The
air
was
filled
with
the
sweet
scent
of
blooming
flowers.
It
was
a
peaceful
and
serene
scene,
perfect
for
a
quiet
evening
stroll.


¿Qué puedes observar sobre los trozos devueltos? Si vieras cada trozo de manera individual, ¿podrías entender el significado semántico?

### 4.2: Chunking dividiendo las oraciones

#### Tarea #5 del Estudiante: 

Agrega código en la celda siguiente. Utiliza la biblioteca `spaCy` y específicamente la función `sents` para dividir el texto en trozos.

In [11]:
text = "Today was a fun day. I had lots of ice cream. I also met my best friend Sally and we played together at the new playground."

# Agrega tu código aquí # Buscar SpaCy y sents... en spaCy exemplos.... 
nlp_model = spacy.load("en_core_web_sm")
doc = nlp_model(text)
sents = list(doc.sents)

sents


[Today was a fun day.,
 I had lots of ice cream.,
 I also met my best friend Sally and we played together at the new playground.]

¿Los resultados son mejores que el método en 4.1? La biblioteca `spaCy` ayuda a dividir el texto en oraciones individuales. Esto puede ser útil cuando intentas hacer resúmenes de texto. Puedes clasificar las oraciones individuales y usar los mejores resultados en el resumen.

### 4.3: Chunking con superposición de oraciones

#### Tarea #6 del Estudiante: 

Ejecuta el código mostrado a continuación para ver otro ejemplo de chunking. Como verás, el significado semántico se mantiene. En otras palabras, se conserva el contexto entre las oraciones. Esto es especialmente importante cuando estás buscando datos para obtener resultados relevantes o cuando estás resumiendo un fragmento de texto. Es importante capturar las relaciones entre las oraciones.

In [12]:
#SOLO EJECUTAR...

text = "The sun was setting over the horizon, casting a warm glow over the landscape. Birds chirped in the trees, and a gentle breeze rustled the leaves. In the distance, a herd of deer grazed in a meadow. The air was filled with the sweet scent of blooming flowers. It was a peaceful and serene scene, perfect for a quiet evening stroll."
doc = nlp(text)

sentences = list(doc.sents)
overlap = 1
chunks =[]

for i in range(len(sentences) - overlap):
    chunk = sentences[i : i + overlap + 1]
    chunks.append(chunk)

for chunk in chunks:
    print([sent.text for sent in chunk])

['The sun was setting over the horizon, casting a warm glow over the landscape.', 'Birds chirped in the trees, and a gentle breeze rustled the leaves.']
['Birds chirped in the trees, and a gentle breeze rustled the leaves.', 'In the distance, a herd of deer grazed in a meadow.']
['In the distance, a herd of deer grazed in a meadow.', 'The air was filled with the sweet scent of blooming flowers.']
['The air was filled with the sweet scent of blooming flowers.', 'It was a peaceful and serene scene, perfect for a quiet evening stroll.']


### 4.4: Chunking de forma recursiva usando LangChain

#### Tarea #7 del Estudiante: 

Agrega los parámetros requeridos para `RecursiveCharacterSplitter` en la celda siguiente.

In [ ]:


split_text = RecursiveCharacterTextSplitter(
   # Agrega los parámetros aquí
   chunk_size=50,  # Tamaño máximo de cada fragmento (en caracteres)
    chunk_overlap=20,  # Cantidad de caracteres que se superponen entre fragmentos
    separators=["\n\n", "\n", " ", ""]  # Prioridad de separación (párrafos, líneas, espacios, etc.)
)
# Divide el texto en fragmentos utilizando el método `create_documents`
docs = split_text.create_documents([clean_text])

# Visualiza los fragmentos resultantes
for doc in docs:
    print(doc.page_content)
# docs

Formula 1 Power Unit Financial Regulations
Regulations     1     16 August     2022     ©
2022     © 202  2                Issue   1
Issue   1              FORMULA 1   POWER
FORMULA 1   POWER UNIT   FINANCIAL REGULATIONS
REGULATIONS     PUBLISHED ON   16 August     2022
16 August     2022     Issue   1        CONTENTS
1        CONTENTS        Art     CONTENTS
CONTENTS     Page(s)     1.     GENERAL
1.     GENERAL PRINCIPLES
PRINCIPLES     ................................
................................
................................  ........     2
........     2     2.     POWER UNIT
2.     POWER UNIT MANUFACTURER OBLIGATIONS
OBLIGATIONS     ................................
................................  ....     3
....     3     3.     EXCLUSIONS
EXCLUSIONS     ................................
................................
................................
.....................     5     4.
5     4.     ADJUSTMENTS
ADJUSTMENTS     ................................
............

Acabamos de realizar algunas fragmentaciones utilizando Langchain, un framework popular para crear aplicaciones utilizando modelos de lenguaje grandes. En los métodos anteriores, viste varios ejemplos de chunking. Langchain puede ayudar a hacer que el proceso de chunking sea más fácil con algunos de sus métodos. Estos métodos incluyen fragmentos de tamaño fijo, así como la fragmentación recursiva, que acabamos de ver.

Por ejemplo, existe `CharacterTextSplitter` que dividirá el texto dado en un fragmento de tamaño fijo de un tamaño determinado y una superposición de caracteres determinada.

`RecursiveCharacterTextSplitter` divide el texto en fragmentos más pequeños de manera iterativa. Nuevamente, puedes proporcionar el tamaño del fragmento y el recuento de superposición del fragmento.

La segmentación es una técnica importante por muchas razones. Ayuda a evitar el límite de tokens cuando se trabaja con grandes cantidades de datos y también optimiza la respuesta que recibimos del modelo. Encontrar la técnica de segmentación adecuada y el tamaño de fragmento es crucial para recibir respuestas relevantes.

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra una comprensión de los tokens y cómo calcularlos.
* Demuestra una comprensión de la segmentación (chunking) experimentando con diferentes técnicas.
* Sé capaz de entender la importancia de encontrar la solución de segmentación adecuada según si se captura o no el significado semántico.